# Planning to TCP Poses 📐

In the previous notebook we saw how to plan paths between joint configurations.
In this notebook we will explore how planning to TCP poses opens doors to many interesting possibilities, such as:
- 💥 **Filtering** e.g. pregrasp configurations to ensure moving the the grasp pose does not collide with the environment
- 🥇 **Selecting** optimal paths, e.g. the shortest, smoothest or fastest after time parametrization
- 📊 **Ranking** goal joint configurations and planning to them iteratively, selecting the first path that is found. This ranking could be based on the distance of the goal joint configuration to the a desirable joint configuration, e.g the start joint configuration.

**The 4-Step Process**

Let's break down how we approach TCP pose planning:
1. 🧮 **IK Solutions:** Calculate candidate goal joint configurations from the goal TCP pose.
2. 🤔 **Rank & Prioritize (Optional):** Rank those candidates
3. 🗺️ **Plan Paths:** Try planning paths to the candidates.
4. 🏆 **Select the Best (Optional):** Choose the optimal path based on your criteria.






<!-- **Why Planning to TCP Poses Matters**

- ✨ **Flexibility:** A single TCP pose often corresponds to multiple possible joint configurations for a 6-DOF robot.
- 💡 **Strategic choices:** This opens doors! We can filter pre-grasp configurations for collision-free grasps, optimize paths based on smoothness or speed, and more.


However, in this notebook we will show how planning to TCP poses opens interesting opportunities.

A key fact to know, is that for a given TCP pose in the workspace of a 6-DOF robot, there will be several joint configurations that can achieve that pose. 
For some robots, such as the universal robots we use at our lab, you can analytically calculate the inverse kinematics to find all the joint configurations that can achieve a given TCP pose.

This finite set of possible goal configurations provide an interesting opportunity.
For example, if the TCP pose we plan to is a pregrasp pose from which we want to move linearly to a grasp pose, we can filter the pregrasp joint configurations to only select the from which we can move to the grasp pose without colliding with the environment.

Another possiblity with multiple joint configurations is that we can plan paths to each of them and then select the best path according to some criteria. For example, we might want to select the shortest path, or the smoothest path, or the path that can be executed the fastest after time parametrization.

However, if any path is good enough, and we care mostly about planning time, we can rank the goal joint configurations, and plan to them iteratively, and select the first path that is found. This ranking could for example be based on the distance goal joint configuration to the a desirable joint configuration .e.g the start joint configuration.


The way we achieve planning to TCP poses is by using a four-step process:
1. Calculate the inverse kinematics of the goal TCP pose to find the candidate goal joint configurations.
2. Optional: rank the goal joint configurations
3. Attempt to plan a path to the goal joint configuration(s)
4. Optional: select the best path -->

## 1. Setting up the scene & collision checker 🏗️️

For more details about this, see the [`airo-drake`](https://github.com/airo-ugent/airo-drake) notebooks.


In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import SingleArmScene, add_floor, add_manipulator, add_meshcat, finish_build
from pydrake.planning import SceneGraphCollisionChecker

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur3e", "robotiq_2f_85", static_gripper=True)
add_floor(robot_diagram_builder)
robot_diagram, context = finish_build(robot_diagram_builder)

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)


collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[scene.arm_index, scene.gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

collision_checker.CheckConfigCollisionFree(np.zeros(6))

## 2. Creating the motion planner 🧭

To enable planning to TCP poses, we need to provide an additional argument to the `SingleArmOmplPlanner` constructor, namely an `inverse_kinematics_fn` that returns a list of joint configurations for a given TCP pose.

In this notebook we will be using the analytical inverse kinematics from the [`ur-analytic-ik`](https://github.com/Victorlouisdg/ur-analytic-ik) Python package.

In [ ]:
from ur_analytic_ik import ur3e
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from airo_drake import visualize_frame

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175  # 175 mm in z


def inverse_kinematics_fn(tcp_pose: HomogeneousMatrixType) -> list[JointConfigurationType]:
    solutions = ur3e.inverse_kinematics_with_tcp(tcp_pose, tcp_transform)
    return solutions


goal_pose = np.identity(4)

X = np.array([-1.0, 0.0, 0.0])
Y = np.array([0.0, 1.0, 0.0])
Z = np.array([0.0, 0.0, -1.0])
top_down_orientation = np.column_stack([X, Y, Z])
goal_pose[:3, :3] = top_down_orientation
goal_pose[:3, 3] = [0.2, 0.0, 0.0]

visualize_frame(scene.meshcat, "goal_pose", goal_pose)

solutions = inverse_kinematics_fn(goal_pose)

# with np.printoptions(precision=3, suppress=True):
for solution in solutions:
    print(solution)


In [ ]:
from airo_drake import animate_joint_configurations

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, solutions)